# Concurency with futures

For I/O bound work that we expect to be used afterwards (in future) we ban submit a work and do some other work meanwhile and collect later on. Requesting result actually joins the thread and assings te resulting data.

In [4]:
import time
from concurrent.futures import ThreadPoolExecutor as PoolExecutor


def do_work(sleep_secs: float = 10.0) -> str:
    time.sleep(sleep_secs)
    return "foo"


def wait_for_future() -> None:
    print("-------- Wait for future --------")
    start_time = time.time()

    with PoolExecutor() as executor:
        future = executor.submit(do_work, sleep_secs=5.0)
        print("future created", "|", time.time() - start_time)

        print("waiting for future...", "|", time.time() - start_time)
        result = future.result()
        print("future result:", result, "|", time.time() - start_time)


def get_future_after() -> None:
    print("--------Get future after --------")
    start_time = time.time()

    with PoolExecutor() as executor:
        future = executor.submit(do_work, sleep_secs=5.0)
        print("future created", "|", time.time() - start_time)

        print("doing other things...", "|", time.time() - start_time)
        time.sleep(10.0)

        result = future.result()
        print("future result:", result, "|", time.time() - start_time)


# ----------------------------------------------------------------

wait_for_future()
get_future_after()

-------- Wait for future --------
future created | 0.0005514621734619141
waiting for future... | 0.0006067752838134766
future result: foo | 5.022968292236328
--------Get future after --------
future created | 0.0002493858337402344
doing other things... | 0.0002949237823486328
future result: foo | 10.05036473274231


## For CPU intensive work one can use ProcessPoolExecutor
Distribution of work is usually done by mapping work. Under the hood multiprocessing is being used.

In [3]:
import time
from concurrent.futures import ProcessPoolExecutor as PoolExecutor
from functools import partial


def do_work(sleep_secs: float, i: int) -> str:
    time.sleep(sleep_secs)
    return f"foo-{i}"



start_time = time.time()
with PoolExecutor() as executor:
    results_gen = executor.map(partial(do_work, 3.0), range(1, 10))
    print("map results:", list(results_gen), "|", time.time() - start_time)

map results: ['foo-1', 'foo-2', 'foo-3', 'foo-4', 'foo-5', 'foo-6', 'foo-7', 'foo-8', 'foo-9'] | 3.032550096511841


## Resources
- Excelent documentation: [python docs](https://docs.python.org/3.10/library/concurrent.futures.html)

